In [ ]:
import google.generativeai as genai
import os
import json # 用於處理函式呼叫的結果
from datetime import datetime, timedelta
import pandas as pd

In [ ]:
# --- 1. 設定您的 Gemini API Key ---
# 建議將您的 API Key 設定為環境變數，以保護其安全。
# 您也可以直接在這裡寫入，但這不推薦用於生產環境。
# 您可以在 Google AI Studio 取得您的 API Key: https://aistudio.google.com/
#
# 要設定環境變數 (Linux/macOS):
# export GOOGLE_API_KEY="您的_API_金鑰"
#
# 要設定環境變數 (Windows Command Prompt):
# set GOOGLE_API_KEY="您的_API_金鑰"
#
# 要設定環境變數 (Windows PowerShell):
# $env:GOOGLE_API_KEY="您的_API_金鑰"

try:
    genai.configure(api_key=os.environ["GOOGLE_API_KEY"])
except KeyError:
    print("錯誤：請設定 GOOGLE_API_KEY 環境變數。")
    print("您可以從 Google AI Studio 取得您的金鑰：https://aistudio.google.com/")
    exit()


In [ ]:
# # --- 2. 輔助函數：選擇可用的模型 ---
def get_supported_model(method="generateContent"):
    """
    列出所有支援指定方法 (例如 'generateContent') 的模型，並回傳第一個。
    優先選擇 Flash 或 Pro 模型。
    """
    print(f"\n正在查詢支援 '{method}' 方法的可用模型...")
    found_models = []
    for m in genai.list_models():
        if method in m.supported_generation_methods:
            found_models.append(m)

    for m in found_models:
        if "gemini-1.5-flash" in m.name:
            print(f"找到並選用 Flash 模型：{m.name} (顯示名稱: {m.display_name})")
            return m.name
    for m in found_models:
        if "gemini-1.5-pro" in m.name:
            print(f"找到並選用 Pro 模型：{m.name} (顯示名稱: {m.display_name})")
            return m.name
    
    if found_models:
        print(f"找到並選用其他可用模型：{found_models[0].name} (顯示名稱: {found_models[0].display_name})")
        return found_models[0].name

    raise ValueError(f"錯誤：找不到任何支援 '{method}' 方法的模型。")

In [ ]:
# --- 3. 模擬數據生成函式 (假爬蟲 或是 模擬資料庫) ---
def generate_fake_weekly_contest_df():
    """生成包含假的 LeetCode Weekly Contest 資訊的 Pandas DataFrame。"""
    
    data = {
        '名稱': [
            "假 Weekly Contest 400",
            "假 Biweekly Contest 120",
            "假 Weekly Contest 399",
            "假 Weekly Contest 398"
        ],
        '開始時間': [
            (datetime.now() + timedelta(days=2, hours=10, minutes=30)).strftime('%Y-%m-%d %H:%M:%S'),
            (datetime.now() + timedelta(days=9, hours=20, minutes=0)).strftime('%Y-%m-%d %H:%M:%S'),
            (datetime.now() - timedelta(days=7, hours=15, minutes=0)).strftime('%Y-%m-%d %H:%M:%S'),
            (datetime.now() - timedelta(days=14, hours=15, minutes=0)).strftime('%Y-%m-%d %H:%M:%S')
        ],
        '連結': [
            "https://leetcode.com/contest/fake-weekly-contest-400/",
            "https://leetcode.com/contest/fake-biweekly-contest-120/",
            "https://leetcode.com/contest/fake-weekly-contest-399/",
            "https://leetcode.com/contest/fake-weekly-contest-398/"
        ]
    }
    return pd.DataFrame(data)

def generate_fake_daily_challenge_df():
    """生成包含假的 LeetCode Daily Challenge 資訊的 Pandas DataFrame。"""
    data = {
        '題目': [
            "模擬每日挑戰題目 - 1001. Two Sum (Daily)",
            "模擬每日挑戰題目 - 1002. Add Two Numbers (Daily)",
            "模擬每日挑戰題目 - 1003. Longest Substring Without Repeating Characters (Daily)"
        ],
        '難度': [
            "Easy",
            "Medium",
            "Hard"
        ],
        '連結': [
            "https://leetcode.com/problems/fake-two-sum/",
            "https://leetcode.com/problems/fake-add-two-numbers/",
            "https://leetcode.com/problems/fake-longest-substring-without-repeating-characters/"
        ]
    }
    return pd.DataFrame(data)


In [9]:
# --- 4. 核心工具函式 (供 Gemini 呼叫) ---

def get_leetcode_daily_info(date: str = None):
    """
    根據給定日期獲取 LeetCode 每日挑戰資訊。
    Args:
        date (str): 查詢日期，格式為 YYYY-MM-DD。如果未提供，則使用當前日期。
    Returns:
        dict: 包含每日挑戰資訊的字典。
    """
    if date is None:
        query_date = datetime.now()
        query_date_str = query_date.strftime('%Y-%m-%d')
    else:
        try:
            query_date = datetime.strptime(date, '%Y-%m-%d')
            query_date_str = date
        except ValueError:
            return {"error": "日期格式錯誤，請使用 YYYY-MM-DD 格式。", "date": date}

    print(f"---- 程式正在執行 get_leetcode_daily_info({query_date_str}) ----")
    fake_df = generate_fake_daily_challenge_df()
    
    # 從假數據中選擇一個作為指定日期的回傳
    # 這裡簡單地根據日期奇偶性來回傳不同數據，模擬不同的每日挑戰
    if query_date.day % 2 == 0:
        info = fake_df.iloc[0].to_dict() # 偶數日給第一筆
    else:
        info = fake_df.iloc[1].to_dict() # 奇數日給第二筆
    
    info['日期'] = query_date_str # 添加日期信息
    return info


def get_leetcode_weekly_info(date: str = None):
    """
    根據給定日期獲取 LeetCode 每週比賽資訊。
    Args:
        date (str): 查詢日期，格式為 YYYY-MM-DD。如果未提供，則使用當前日期。
    Returns:
        dict: 包含每週比賽資訊的字典。
    """
    if date is None:
        query_date = datetime.now()
        query_date_str = query_date.strftime('%Y-%m-%d')
    else:
        try:
            query_date = datetime.strptime(date, '%Y-%m-%d')
            query_date_str = date
        except ValueError:
            return {"error": "日期格式錯誤，請使用 YYYY-MM-DD 格式。", "date": date}

    print(f"---- 程式正在執行 get_leetcode_weekly_info({query_date_str}) ----")
    fake_df = generate_fake_weekly_contest_df()

    # 從假數據中選擇一個作為指定日期的回傳
    # 這裡模擬根據日期判斷是過去的比賽還是未來的比賽
    if query_date > datetime.now():
        info = fake_df.iloc[0].to_dict() # 未來日期給第一個即將到來的比賽
    elif query_date.weekday() in [5, 6]: # 週六或週日，假設是比賽日
        info = fake_df.iloc[1].to_dict() # 比賽日給第二個
    else:
        info = fake_df.iloc[2].to_dict() # 其他日期給一個過去的比賽
    
    info['日期'] = query_date_str # 添加日期信息
    return info

def is_holiday(date: str):
    """
    判斷給定日期是否是假日（模擬判斷）。
    Args:
        date (str): 查詢日期，格式為 YYYY-MM-DD。
    Returns:
        dict: 包含 'is_holiday' (bool) 和 'reason' (str) 的字典。
    """
    try:
        query_date = datetime.strptime(date, '%Y-%m-%d')
    except ValueError:
        return {"error": "日期格式錯誤，請使用 YYYY-MM-DD 格式。", "date": date}

    print(f"---- 程式正在執行 is_holiday({date}) ----")

    # 模擬假日判斷邏輯
    # 預設週末為假日
    if query_date.weekday() in [5, 6]:  # 5是週六, 6是週日
        return {"is_holiday": True, "reason": "週末", "date": date}
    
    # 模擬特定節日
    # 假設 2025/1/1 是元旦
    if query_date.month == 1 and query_date.day == 1:
        return {"is_holiday": True, "reason": "元旦", "date": date}
    # 假設 2025/10/10 是國慶日
    if query_date.month == 10 and query_date.day == 10:
        return {"is_holiday": True, "reason": "國慶日", "date": date}
    
    return {"is_holiday": False, "reason": "工作日", "date": date}


In [ ]:
# --- 5. 定義工具的描述 (Tool Definition) ---
try:
    model_to_use = "models/gemini-1.5-flash-latest" 
    try:
        test_model = genai.get_model(model_to_use)
        if "generateContent" not in test_model.supported_generation_methods:
             raise ValueError(f"模型 {model_to_use} 不支援 generateContent。")
        print(f"\n嘗試使用模型: {model_to_use}")
    except Exception:
        print(f"\n模型 {model_to_use} 不可用或不支援，正在尋找其他可用模型...")
        model_to_use = get_supported_model(method="generateContent")

except ValueError as e:
    print(e)
    exit()

tools = [
    {
        "function_declarations": [
            {
                "name": "get_leetcode_daily_info",
                "description": "根據給定日期獲取 LeetCode 每日挑戰資訊。如果未提供日期，則回傳當前日期的資訊。",
                "parameters": {
                    "type": "OBJECT",
                    "properties": {
                        "date": {
                            "type": "string",
                            "description": "查詢日期，格式為 YYYY-MM-DD。例如：2025-06-26"
                        }
                    },
                    "required": []
                },
            },
            {
                "name": "get_leetcode_weekly_info",
                "description": "根據給定日期獲取 LeetCode 每週比賽資訊。如果未提供日期，則回傳當前日期的資訊。",
                "parameters": {
                    "type": "OBJECT",
                    "properties": {
                        "date": {
                            "type": "string",
                            "description": "查詢日期，格式為 YYYY-MM-DD。例如：2025-06-29"
                        }
                    },
                    "required": []
                },
            },
            {
                "name": "is_holiday",
                "description": "判斷給定日期是否為假日。",
                "parameters": {
                    "type": "OBJECT",
                    "properties": {
                        "date": {
                            "type": "string",
                            "description": "查詢日期，格式為 YYYY-MM-DD。例如：2025-01-01"
                        }
                    },
                    "required": ["date"] # 日期是必填參數
                },
            }
        ]
    }
]


嘗試使用模型: models/gemini-1.5-flash-latest


In [27]:
# 初始化模型，並帶上工具
leetcode_tool_model = genai.GenerativeModel(
    model_name=model_to_use,
    tools=tools
)

# --- 6. 與 Gemini 模型互動並處理函式呼叫 ---

chat = leetcode_tool_model.start_chat(enable_automatic_function_calling=False) # 關閉自動呼叫，讓我們手動控制

print("您可以問：")
print("- '今天的 LeetCode 每日挑戰是什麼？'")
print("- '2025年1月1日的 LeetCode 挑戰是什麼？那天的比賽呢？'")
print("- '2025年6月29日是假日嗎？那天的 LeetCode 資訊是什麼？'")


# user_message = '今天的題目是甚麼?'
user_message = '2025年6月29日是假日嗎？那天的 LeetCode 資訊是什麼？'
print(f"\n使用者輸入: {user_message}")
response = chat.send_message(user_message)

try:
    # 檢查 response 是否包含一個或多個 function_call
    if response.candidates and response.candidates[0].content and \
        response.candidates[0].content.parts and \
        any(hasattr(part, 'function_call') for part in response.candidates[0].content.parts):

        requested_date = None
        is_holiday_result = None

        # First pass: Identify if is_holiday is requested and get the date
        for part in response.candidates[0].content.parts:
            if hasattr(part, 'function_call'):
                function_call = part.function_call
                function_name = function_call.name
                function_args = function_call.args
                
                if function_name == "is_holiday":
                    requested_date = function_args.get("date")
                    if requested_date:
                        print(f"--- 模型請求呼叫函式：{function_name}，參數：{function_args} ---")
                        is_holiday_result = is_holiday(**function_args)
                        print(f"--- 函式執行結果：{is_holiday_result} ---")
                        # Add is_holiday result immediately
                        # chat.send_message(FunctionResponse(name=function_name, response=is_holiday_result))
                        break

        # Determine which functions to call based on is_holiday result or user prompt
        functions_to_call = []
        
        # Always call get_leetcode_daily_info
        if requested_date:
            functions_to_call.append({"name": "get_leetcode_daily_info", "args": {"date": requested_date}})
        else:
            # If no specific date was requested by model (e.g., "今天的挑戰")
            functions_to_call.append({"name": "get_leetcode_daily_info", "args": {}})

        # If it's a holiday, also call get_leetcode_weekly_info
        if is_holiday_result and is_holiday_result.get("is_holiday"):
            if requested_date:
                functions_to_call.append({"name": "get_leetcode_weekly_info", "args": {"date": requested_date}})
            else:
                functions_to_call.append({"name": "get_leetcode_weekly_info", "args": {}})

        # Execute the determined functions and collect responses
        function_responses = []
        for func_info in functions_to_call:
            print(f"--- 程式根據條件邏輯，決定呼叫函式：{func_info['name']}，參數：{func_info['args']} ---")
            function_name = func_info["name"]
            function_args = func_info["args"]
            
            print(f"--- 程式根據條件邏輯，呼叫函式：{function_name}，參數：{function_args} ---")
            
            result = None
            if function_name == "get_leetcode_daily_info":
                result = get_leetcode_daily_info(**function_args)
            elif function_name == "get_leetcode_weekly_info":
                result = get_leetcode_weekly_info(**function_args)
            print(f"--- 函式執行結果：{result} ---")

    else:
        # 如果沒有 function_call，表示模型直接產生了文字回覆
        print("\nGemini 模型回覆:")
        print(response.text)
except Exception as e:
    print(f"處理回應時發生錯誤: {e}")
    try:
        print("\n原始模型回應結構:")
        print(response)
    except Exception:
        print("無法取得模型回應物件。")


您可以問：
- '今天的 LeetCode 每日挑戰是什麼？'
- '2025年1月1日的 LeetCode 挑戰是什麼？那天的比賽呢？'
- '2025年6月29日是假日嗎？那天的 LeetCode 資訊是什麼？'

使用者輸入: 2025年6月29日是假日嗎？那天的 LeetCode 資訊是什麼？
--- 模型請求呼叫函式：is_holiday，參數：<proto.marshal.collections.maps.MapComposite object at 0x000002188289BED0> ---
---- 程式正在執行 is_holiday(2025-06-29) ----
--- 函式執行結果：{'is_holiday': True, 'reason': '週末', 'date': '2025-06-29'} ---
--- 程式根據條件邏輯，決定呼叫函式：get_leetcode_daily_info，參數：{'date': '2025-06-29'} ---
--- 程式根據條件邏輯，呼叫函式：get_leetcode_daily_info，參數：{'date': '2025-06-29'} ---
---- 程式正在執行 get_leetcode_daily_info(2025-06-29) ----
--- 函式執行結果：{'題目': '模擬每日挑戰題目 - 1002. Add Two Numbers (Daily)', '難度': 'Medium', '連結': 'https://leetcode.com/problems/fake-add-two-numbers/', '日期': '2025-06-29'} ---
--- 程式根據條件邏輯，決定呼叫函式：get_leetcode_weekly_info，參數：{'date': '2025-06-29'} ---
--- 程式根據條件邏輯，呼叫函式：get_leetcode_weekly_info，參數：{'date': '2025-06-29'} ---
---- 程式正在執行 get_leetcode_weekly_info(2025-06-29) ----
--- 函式執行結果：{'名稱': '假 Weekly Contest 400', '開始時間': '2025-06-29 09